## Examine a table using CQL
(Jupyter notebook feature)
Use the %%Cql Magic to prefix your CQL.

In [ ]:
%%showschema cloud1.ldomain

In [8]:
%%Cql select * from cloud1.ldomain limit 5

domain,total
decoybetty.com,1
thegreenwineguide.com,108
trotmanautomotivegroup.com,1
yakultusa.com,2
greatlakesgelatin.com,3


## Creating RDDs from Cassandra Tables
* Can add a where clause to push down filter
* Creates and RDD of CassandraRow objects
* .as will map it to a case class or tuples for ease of use


In [9]:
val tracks = sc.cassandraTable("cloud1","ldomain")
tracks

com.datastax.spark.connector.rdd.CassandraTableScanRDD[com.datastax.spark.connector.CassandraRow] = CassandraTableScanRDD[3] at RDD at CassandraRDD.scala:15

In [10]:
tracks.first


com.datastax.spark.connector.CassandraRow = CassandraRow{domain: decoybetty.com, total: 1}

### get the album and track in a tuple.  This is the new syntax:

In [11]:
val albumTracks = sc.cassandraTable[(String,Integer)]("cloud1",
"ldomain").select("domain","total")
albumTracks

com.datastax.spark.connector.rdd.CassandraTableScanRDD[(String, Integer)] = CassandraTableScanRDD[6] at RDD at CassandraRDD.scala:15

The first 10 rows as tuples ....

In [12]:
albumTracks.take(10) foreach println

(decoybetty.com,1)
(thegreenwineguide.com,108)
(trotmanautomotivegroup.com,1)
(yakultusa.com,2)
(greatlakesgelatin.com,3)
(dgs.no,1)
(brbspa.it,2)
(wxcjpj.com,5)
(aptoide.com,32)
(online-language-translations.co.uk,1)


### Create RDDs from Cassandra Tables and return an RDD of case class objects
.as() will map the rdd to a case class


In [13]:
case class Track(domain: String,
total:Int)

In [14]:
val tracks = sc.cassandraTable[Track]("cloud1","ldomain")
tracks

com.datastax.spark.connector.rdd.CassandraTableScanRDD[Track] = CassandraTableScanRDD[8] at RDD at CassandraRDD.scala:15

In [15]:
tracks take 5 foreach println

Track(decoybetty.com,1)
Track(thegreenwineguide.com,108)
Track(trotmanautomotivegroup.com,1)
Track(yakultusa.com,2)
Track(greatlakesgelatin.com,3)


## Some other useful actions ...
* first – same as take(1)(0)
* collect – bring everything back to the caller as a scala array
* saveToCassandra
* count


In [16]:
tracks.first

Track = Track(decoybetty.com,1)

In [17]:
tracks.count

Long = 1620382

## Some Typical Transformations
filter, map, distinct

Show tracks from 1989

In [28]:
tracks.filter(x => x.total > 2000000).take(20).foreach(println)


Track(oscaro.es,3426581)
Track(fvd.de,2931849)
Track(mihav.com,7136664)
Track(tanners-wines.co.uk,103532249)
Track(industrialmanuals.com,19220462)
Track(hbees.com,2127802)
Track(usermed.com,6366922)
Track(fotki.com,5012496)
Track(efaucets.com,22238160)
Track(alifmultimedia.com,2034081)
Track(chinesemedicinetimes.com,12577279)
Track(truthinshredding.com,9018942)
Track(SharjahYellowPagesOnline.com,2170827)
Track(surfline.dk,2746163)
Track(it-gnoth.de,4587215)
Track(thurstontalk.com,8916498)
Track(kupimauto.sk,14752346)
Track(callcenterbestpractices.com,18596324)
Track(bogg.com,2833131)
Track(uni-trier.de,2474913)


In [ ]:
tracks.collect() take 10 foreach println

**This can also be accomplished with a .where function on the cassandraTable to push the work into Cassandra**

map the cassandra table to 2-tuples 

In [30]:
tracks.map(x =>(x.domain, x.total)).take(5).foreach(println)

(decoybetty.com,1)
(thegreenwineguide.com,108)
(trotmanautomotivegroup.com,1)
(yakultusa.com,2)
(greatlakesgelatin.com,3)


Combine operations into a single graphe or even a single statement

In [ ]:
tracks.filter(x => x.album_year == 1990).
map(x => (x.album_title, x.track_title)).
take(5).foreach(println)


## Pair RDDs – Special operations on RDD of  2-Tuples
* Think of each tuple as (Key,Value)
* countByKey
* groupByKey
* reduceByKey


In [ ]:
val albumTracks = tracks.map(t => (t.album_title, t.track_title))

How many tracks in each album?

In [ ]:
val trackTitles = albumTracks.countByKey
trackTitles

Why not sort the results descending? toList turns the map into a list of tuples and sort by the negative of the count

## Top 10 List


In [ ]:
albumTracks.countByKey.toList.sortBy( t => -t._2 ) take 10 foreach println

In [ ]:
tracks.filter(_.album_title == "Greatest Hits").collect foreach println

In [ ]:
var x:Option[Int] = Some(5)

In [ ]:
x

In [ ]:
x = None

In [ ]:
x.orElse(Some(0))

In [ ]:
tracks.filter(_.album_title == "Greatest Hits").saveAsTextFile("cfs:///tmp/tracks2")